# Homework 3: Algorithms for DNA sequencing

In [3]:
import numpy as np
import functions
from functions import readf

<b>Question 1:</b><br>
What is the edit distance of the best match between pattern <i>GCTGATCGATCGTACG</i> and the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [1]:
def edit_recursive(x,y):
    matrix=np.zeros((len(x)+1,len(y)+1))
    matrix[0,0]=0 #Setting empty coordinates to 0
    for i in range(1,len(x)+1):
        matrix[i,0]=matrix[i-1,0]+1
    for i in range(1,len(x)+1):
        for j in range(1,len(y)+1):
            dist_last=0 if x[i-1]==y[j-1] else 1
            matrix[i,j]=min(matrix[i-1,j-1]+dist_last,matrix[i-1,j]+1, matrix[i,j-1]+1)
    return min(matrix[-1])

In [4]:
text1=next(iter(readf('chr1.GRCh38.excerpt.fasta').values()))

In [9]:
patt='GCTGATCGATCGTACG'
print('The edit distances of the best match is ',edit_recursive(patt,text1))

The edit distances of the best match is  3.0


<b>Question 2: </b><br>
What is the edit distance of the best match between pattern <i>GATTTACCAGATTGAG</i> and the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [11]:
patt='GATTTACCAGATTGAG'
print('The edit distances of the best match is ',edit_recursive(patt,text1))

The edit distances of the best match is  2.0


<b>Question 3:</b><br>
In a practical, we saw a function for finding the longest exact overlap (suffix/prefix match) between two strings.<br>
Say we are concerned only with overlaps that (a) are exact matches (no differences allowed), and (b) are at least k bases long. To make an overlap graph, we could call overlap(a, b, min_length=k) on every possible pair of reads from the dataset.  Unfortunately, that will be very slow!

Consider this: Say we are using k=6, and we have a read a whose length-6 suffix is <i>GTCCTA</i>.  Say <i>GTCCTA</i> does not occur in any other read in the dataset.  In other words, the 6-mer <i>GTCCTA</i> occurs at the end of read a and nowhere else.  It follows that a's suffix cannot possibly overlap the prefix of any other read by 6 or more characters.

Put another way, if we want to find the overlaps involving a suffix of read \verb|a|a and a prefix of some other read, we can ignore any reads that don't contain the length-k suffix of a.  This is good news because it can save us a lot of work!

Here is a suggestion for how to implement this idea.  You don't have to do it this way, but this might help you.  Let every k-mer in the dataset have an associated Python set object, which starts out empty.  We use a Python dictionary to associate each k-mer with its corresponding set. (1) For every k-mer in a read, we add the read to the set object corresponding to that k-mer.  If our read is <i>GATTA</i> and k=3, we would add <i>GATTA</i> to the set objects for <i>GAT</i>, <i>ATT</i> and <i>TTA</i>.  We do this for every read so that, at the end, each set contains all reads containing the corresponding k-mer.  (2) Now, for each read a, we find all overlaps involving a suffix of a.  To do this, we take a's length-k suffix, find all reads containing that k-mer (obtained from the corresponding set) and call overlap(a, b, min_length=k) for each.

The most important point is that we do not call overlap(a, b, min_length=k) if b does not contain the length-k suffix of a.

Download and parse the read sequences from the provided Phi-X FASTQ file. We'll just use their base sequences, so you can ignore read names and base qualities.  Also, no two reads in the FASTQ have the same sequence of bases.  This makes things simpler.

https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq

Next, find all pairs of reads with an exact suffix/prefix match of length at least 30. Don't overlap a read with itself; if a read has a suffix/prefix match to itself, ignore that match.  Ignore reverse complements.

<u>Hint 1:</u> Your function should not take much more than 15 seconds to run on this 10,000-read dataset, and maybe much less than that.  (Our solution takes about 3 seconds.) If your function is much slower, there is a problem somewhere.

<u>Hint 2:</u> Remember not to overlap a read with itself. If you do, your answers will be too high.

<u>Hint 3:</u> You can test your implementation by making up small examples, then checking that:<br>&nbsp;&nbsp;&nbsp;(a) your implementation runs quickly, and<br>&nbsp;&nbsp;&nbsp;(b) you get the same answer as if you had simply called overlap(a, b, min_length=k) on every pair of reads.<br>We also have provided a couple examples you can check against.

Picture the overlap graph corresponding to the overlaps just calculated.  How many edges are in the graph?  In other words, how many distinct pairs of reads overlap?

In [16]:
def overlap(a,b,minlength=3): #Find if prefix of 'b' is in suffix of 'a'
    start=0
    while True:
        start = a.find(b[:minlength],start)
        if start==-1:
            return 0
        if b.startswith(a[start:]): ###Only need the space where b starts
            return len(a)-start
        start+=1

In [17]:
from collections import defaultdict
def fast_overlap(reads,kmer=6,minlength=3):
    count=0
    dict_kmers=defaultdict(set)
    for read in reads:
        for t in range(len(read)-kmer+1):
            dict_kmers[read[t:t+kmer]].add(read)
 #This is a dictionary with key=kmer and value=entire read
    overlap_dict=defaultdict(set)
    for read in reads:
        suffs=read[-kmer:] #Get the last kmer/ suffix of length k from the read
        read_suffs=dict_kmers[suffs] #Get all the reads that have this suffix somewhere in their list
        for suff in read_suffs: 
            if read!=suff:#If the original read is not equal to the current read selected:
                olen=overlap(read,suff,minlength)
                if olen>0:
                    count+=1
                    overlap_dict[(read,suff)].add(olen)
    return overlap_dict,count


In [18]:
from functions import read_q

In [19]:
seq,_=read_q('ERR266411_1.for_asm.fastq')

In [20]:
dict_overlap,count_olen=fast_overlap(seq,kmer=30,minlength=30)

In [22]:
print('No. of edges in the graph:',count_olen)

No. of edges in the graph: 904746


<b>Question 5.</b><br>
Picture the overlap graph corresponding to the overlaps computed for the previous question. How many nodes in this graph have at least one outgoing edge?  (In other words, how many reads have a suffix involved in an overlap?)

<b>Ans: </b>In the dict_overlap keys, we have a tuple of two reads: Read 1 as the primary read from all samples and Read 2 as another read which has a sub-string that matches the suffix of the read. Hence, each tuple can be considered as an outgoing edge from Read 1 to Read 2. To solve this, we need the unqiue count of Read 1s.

In [25]:
outs=len(set([k1 for k1,k2 in dict_check.keys()]))
print('The total number of nodes having atleast one outgoing edge is:',outs)

The total number of nodes having atleast one outgoing edge is: 7161
